In [118]:
import os
import pandas as pd

In [119]:
baseDir = '../data/'

In [120]:
li = []
for fileName in os.listdir(baseDir)[:2]:
    currentFilePath = os.path.join(baseDir, fileName)
    df1 = pd.read_csv(currentFilePath, header=None)
    df1[0] = fileName
    li.append(df1)

In [121]:
df = pd.concat(li)
df[4].unique()

array([1, 2, 3, 4, 5, 6, 7, 0], dtype=int64)

In [122]:
df[4].value_counts()

7    117179
1     77827
4     49035
3     34654
6     10017
5      7101
2      4418
0       271
Name: 4, dtype: int64

In [123]:
df = df[df[4] != 0]
df[4].unique()

array([1, 2, 3, 4, 5, 6, 7], dtype=int64)

In [124]:
df.tail()

,0,1,2,3,4
137995,2.csv,2150,2316,1979,7
137996,2.csv,2142,2314,1985,7
137997,2.csv,2139,2301,1992,7
137998,2.csv,2136,2315,1990,7
137999,2.csv,2134,2320,1982,7


In [125]:
df.rename(columns = {0:'filename', 1:'x_acc', 2:'y_acc', 3:'z_acc', 4:'label'}, inplace = True) 


In [126]:
from sklearn import linear_model
from sklearn import model_selection
from sklearn import utils
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn import tree
from sklearn.feature_selection import RFECV
from sklearn import feature_selection
from sklearn import naive_bayes
from sklearn import neighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import GridSearchCV

min_max_scaler = MinMaxScaler()
robust_scaler = RobustScaler()

df[df.columns[1:-1]] = robust_scaler.fit_transform(df[df.columns[1:-1]])



def modelstats1(Xtrain,Xtest,ytrain,ytest):
    stats=[]
#     modelnames=["DecisionTree","KNN","NB"]
    modelnames = ["KNN"]
    models=list()
#     models.append(linear_model.LogisticRegression())
#     models.append(tree.DecisionTreeClassifier())
    models.append(neighbors.KNeighborsClassifier())
#     models.append(naive_bayes.GaussianNB())
    for name,model in zip(modelnames,models):
        if name=="KNN":
            for k in range(17,30,4):
                model = neighbors.KNeighborsClassifier(n_neighbors=k)
#             grid={"n_neighbors":k}
#             grid_obj = model_selection.GridSearchCV(estimator=model,param_grid=grid,scoring="f1_macro")
#             grid_fit =grid_obj.fit(Xtrain,ytrain)
#             model = grid_fit.best_estimator_
                model.fit(Xtrain,ytrain)
                trainprediction=model.predict(Xtrain)
                testprediction=model.predict(Xtest)
                scores=list()
                scores.append(name+"-train")
                scores.append(metrics.accuracy_score(ytrain,trainprediction))
                stats.append(scores)
                scores=list()
                scores.append(name+"-test")
                scores.append(metrics.accuracy_score(ytest,testprediction))
                stats.append(scores)
                print("for k = ",k,"training accuracy = ",metrics.accuracy_score(ytrain,trainprediction),"testing accuracy = ",metrics.accuracy_score(ytest,testprediction))
#             name=name+"("+str(grid_fit.best_params_["n_neighbors"])+")"
#             print(grid_fit.best_params_)
        else:
            model.fit(Xtrain,ytrain)
            
        trainprediction=model.predict(Xtrain)
        testprediction=model.predict(Xtest)
        scores=list()
        scores.append(name+"-train")
        scores.append(metrics.accuracy_score(ytrain,trainprediction))
#         scores.append(metrics.precision_score(ytrain,trainprediction,average='micro'))
#         scores.append(metrics.recall_score(ytrain,trainprediction,average='micro'))
#         scores.append(metrics.roc_auc_score(ytrain,trainprediction))
        stats.append(scores)
        scores=list()
        scores.append(name+"-test")
        scores.append(metrics.accuracy_score(ytest,testprediction))
#         scores.append(metrics.precision_score(ytest,testprediction,average='micro'))
#         scores.append(metrics.recall_score(ytest,testprediction,average='micro'))
#         scores.append(metrics.roc_auc_score(ytest,testprediction))
        stats.append(scores)
    
#     colnames=["MODELNAME","ACCURACY","PRECISION","RECALL","AUC"]
        colnames=["MODELNAME","ACCURACY"]
#         break
    return pd.DataFrame(stats,columns=colnames)

In [127]:
X = df.drop(['filename','label'], axis=1)
y = df['label']

In [128]:
Xtrain,Xtest,ytrain,ytest=model_selection.train_test_split(X,y,test_size=.2,random_state=42)
modelstats1(Xtrain,Xtest,ytrain,ytest)

for k =  17 training accuracy =  0.8233021350298105 testing accuracy =  0.8076340200176528
for k =  21 training accuracy =  0.8216950338074143 testing accuracy =  0.8081003214148916
for k =  25 training accuracy =  0.8202419811477867 testing accuracy =  0.8090329242093693
for k =  29 training accuracy =  0.8187889284881591 testing accuracy =  0.8085999300547904


,MODELNAME,ACCURACY
0,KNN-train,0.823302
1,KNN-test,0.807634
2,KNN-train,0.821695
3,KNN-test,0.808100
4,KNN-train,0.820242
5,KNN-test,0.809033
6,KNN-train,0.818789
7,KNN-test,0.808600
8,KNN-train,0.818789
9,KNN-test,0.808600


In [129]:
modelnames="KNN"
models=list()
#     models.append(linear_model.LogisticRegression())
models.append(tree.DecisionTreeClassifier())
models.append(neighbors.KNeighborsClassifier())
models.append(naive_bayes.GaussianNB())
for name,model in zip(modelnames,models):
    print(name, model)

K DecisionTreeClassifier()
N KNeighborsClassifier()
N GaussianNB()


In [130]:
def printscores(ytest,predicted):
    confmatrix=metrics.confusion_matrix(ytest,predicted)
    print("Confusion matrix:\n",confmatrix)
    print("Accuracy:\n",metrics.accuracy_score(ytest,predicted))

In [131]:
Xtrain,Xtest,ytrain,ytest=model_selection.train_test_split(X,y,test_size=.15,random_state=11)
model = neighbors.KNeighborsClassifier(n_neighbors=17)
model.fit(Xtrain,ytrain)
prediction=model.predict(Xtest)

printscores(ytest,prediction)

Confusion matrix:
 [[11463    89     5    90    10     4    71]
 [  278   212    14    99     4     0    75]
 [    7     1  2727   688    91    52  1616]
 [  106     2   383  5833    60    19   953]
 [   18     0   329   355   199     8   147]
 [    5     1    72   319    25   424   631]
 [  101    10   732   839    40   232 15596]]
Accuracy:
 0.809459309426002


In [132]:
def modelstats1(Xtrain,Xtest,ytrain,ytest):
    stats=[]
    modelnames = ["KNN"]
    models=list()
    models.append(neighbors.KNeighborsClassifier())
    for name,model in zip(modelnames,models):
        if name=="KNN":
            for k in range(17,30,4):
                model = neighbors.KNeighborsClassifier(n_neighbors=k)
                model.fit(Xtrain,ytrain)
                trainprediction=model.predict(Xtrain)
                testprediction=model.predict(Xtest)
                scores=list()
                scores.append(name+"-train")
                scores.append(metrics.accuracy_score(ytrain,trainprediction))
                stats.append(scores)
                scores=list()
                scores.append(name+"-test")
                scores.append(metrics.accuracy_score(ytest,testprediction))
                stats.append(scores)
                print("for k = ",k,"training accuracy = ",metrics.accuracy_score(ytrain,trainprediction),"testing accuracy = ",metrics.accuracy_score(ytest,testprediction))
            
        colnames=["MODELNAME","ACCURACY"]
    return pd.DataFrame(stats,columns=colnames)

In [133]:
Xtrain,Xtest,ytrain,ytest=model_selection.train_test_split(X,y,test_size=.2,random_state=42)
modelstats1(Xtrain,Xtest,ytrain,ytest)

for k =  17 training accuracy =  0.8233021350298105 testing accuracy =  0.8076340200176528
for k =  21 training accuracy =  0.8216950338074143 testing accuracy =  0.8081003214148916
for k =  25 training accuracy =  0.8202419811477867 testing accuracy =  0.8090329242093693
for k =  29 training accuracy =  0.8187889284881591 testing accuracy =  0.8085999300547904


,MODELNAME,ACCURACY
0,KNN-train,0.823302
1,KNN-test,0.807634
2,KNN-train,0.821695
3,KNN-test,0.808100
4,KNN-train,0.820242
5,KNN-test,0.809033
6,KNN-train,0.818789
7,KNN-test,0.808600


In [ ]:
leaf_size = list(range(48,50))
n_neighbors = list(range(28,30))
p=[1,2]
knn = KNeighborsClassifier()
#convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
#Making model
clf = GridSearchCV(knn, hyperparameters, cv=10)
best_model = clf.fit(Xtrain,ytrain)
#Best Hyperparameters Value
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])
#Predict testing set
y_pred = best_model.predict(Xtest)
#Check performance using accuracy
print(accuracy_score(ytest, ypred))
#Check performance using ROC
roc_auc_score(y_test, y_pred)